In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install faiss-cpu

In [ ]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# **Loadng the csv file**

In [ ]:
# Step 1: Load data from the CSV file
def load_data_from_csv(file_path):

    df = pd.read_csv(file_path)
    questions = df['question'].tolist()
    answers = df['answer'].tolist()
    return questions, answers

# **Creating KB**

In [ ]:
def create_healthcare_knowledge_base(questions):
    """
    Takes a list of questions and converts it into a vectorstore for retrieval.
    """
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(questions, embeddings)
    return vectorstore

In [ ]:
csv_file_path = "/content/medquad.csv"

questions, answers = load_data_from_csv(csv_file_path)

knowledge_base = create_healthcare_knowledge_base(questions)

# **Setting up the RetrievalQA chain**

In [ ]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_new_tokens=50
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)
retriever = knowledge_base.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


# **Prompting**

In [ ]:
print("Welcome to the Healthcare Q&A System!")
while True:
    question = input("Please enter your healthcare-related question (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        print("Thank you for using the Healthcare Q&A System!")
        break

    # Use the question for retrieval
    response = qa_chain.invoke(question)
    if "source_documents" in response and response["source_documents"]:
        # Map back to the original answer based on the retrieved question
        retrieved_question = response["source_documents"][0].page_content
        answer_index = questions.index(retrieved_question)
        retrieved_answer = answers[answer_index]
        print("Answer:", retrieved_answer)
    else:
        print("I'm sorry, I couldn't find a relevant answer.")

Welcome to the Healthcare Q&A System!
Please enter your healthcare-related question (or type 'exit' to quit): suggest medicine for fever
Answer: In the United States, Q fever outbreaks have resulted mainly from occupational exposure involving veterinarians, meat processing plant workers, sheep and dairy workers, livestock farmers, and researchers at facilities housing sheep. Prevention and control efforts should be directed primarily toward these groups and environments. 
 
The following measures should be used in the prevention and control of Q fever: 
  
  - Educate the public on sources of infection.  
  - Appropriately dispose of placenta, birth products, fetal membranes, and aborted fetuses at facilities housing sheep and goats.  
  - Restrict access to barns and laboratories used in housing potentially infected animals.   
  - Use appropriate procedures for bagging, autoclaving, and washing of laboratory clothing.  
  - Vaccinate (where possible) individuals engaged in research w